# How to build a tool-using agent with LangChain

This notebook takes you through how to use LangChain to augment an OpenAI model with access to external tools. In particular, you'll be able to create LLM agents that use custom tools to answer user queries.


## What is Langchain?
[LangChain](https://python.langchain.com/en/latest/index.html) is a framework for developing applications powered by language models. Their framework enables you to build layered LLM-powered applications that are context-aware and able to interact dynamically with their environment as agents, leading to simplified code for you and a more dynamic user experience for your customers.

## Why do LLMs need to use Tools?
One of the most common challenges with LLMs is overcoming the lack of recency and specificity in their training data - answers can be out of date, and they are prone to hallucinations given the huge variety in their knowledge base. Tools are a great method of allowing an LLM to answer within a controlled context that draws on your existing knowledge bases and internal APIs - instead of trying to prompt engineer the LLM all the way to your intended answer, you allow it access to tools that it calls on dynamically for info, parses, and serves to customer. 

Providing LLMs access to tools can enable them to answer questions with context directly from search engines, APIs or your own databases. Instead of answering directly, an LLM with access to tools can perform intermediate steps to gather relevant information. Tools can also be used in combination. [For example](https://python.langchain.com/en/latest/modules/agents/agents/examples/mrkl_chat.html), a language model can be made to use a search tool to lookup quantitative information and a calculator to execute calculations.

## Notebook Sections

- **Setup:** Import packages and connect to a Pinecone vector database.
- **LLM Agent:** Build an agent that leverages a modified version of the [ReAct](https://react-lm.github.io/) framework to do chain-of-thought reasoning.
- **LLM Agent with History:** Provide the LLM with access to previous steps in the conversation.
- **Knowledge Base:** Create a knowledge base of "Stuff You Should Know" podcast episodes, to be accessed through a tool.
- **LLM Agent with Tools:** Extend the agent with access to multiple tools and test that it uses them to answer questions.

In [1]:
%load_ext autoreload
%autoreload 2

# Setup

Import libraries and set up a connection to a [Pinecone](https://www.pinecone.io) vector database.

You can substitute Pinecone for any other vectorstore or database - there are a [selection](https://python.langchain.com/en/latest/modules/indexes/vectorstores.html) that are supported by Langchain natively, while other connectors will need to be developed yourself.

In [2]:
# !pip install openai
# !pip install pinecone-client
# !pip install pandas
# !pip install typing
# !pip install tqdm
# !pip install langchain
# !pip install wget

In [3]:
import datetime
import json
import openai
import os
import pandas as pd
import pinecone
import re
from tqdm.auto import tqdm
from typing import List, Union
import zipfile

# Langchain imports
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.prompts import BaseChatPromptTemplate, ChatPromptTemplate
from langchain import SerpAPIWrapper, LLMChain
from langchain.schema import AgentAction, AgentFinish, HumanMessage, SystemMessage
# LLM wrapper
from langchain.chat_models import ChatOpenAI
from langchain import OpenAI
# Conversational memory
from langchain.memory import ConversationBufferWindowMemory
# Embeddings and vectorstore
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone

# Vectorstore Index
index_name = 'podcasts'

/opt/homebrew/anaconda3/envs/openai-cookbook/lib/python3.11/site-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


For acquiring an API key to connect with Pinecone, you can set up a [free account](https://app.pinecone.io/) and store it in the `api_key` variable below or in your environment variables under `PINECONE_API_KEY`

In [4]:
api_key = os.getenv("PINECONE_API_KEY") or "PINECONE_API_KEY"

# find environment next to your API key in the Pinecone console
env = os.getenv("PINECONE_ENVIRONMENT") or "PINECONE_ENVIRONMENT"

print(f"Using Pinecone environment: {env}")
print(f"Using Pinecode API Key: {api_key}")

Using Pinecone environment: asia-northeast1-gcp
Using Pinecode API Key: 4112fb51-d5b8-4288-b289-a89aebf10d74


In [5]:
#api_key="4112fb51-d5b8-4288-b289-a89aebf10d74"
#env="asia-northeast1-gcp"
pinecone.init(api_key=api_key, environment=env)
pinecone.whoami()

WhoAmIResponse(username='3aae44b', user_label='default', projectname='5211ec9')

In [6]:
pinecone.list_indexes()

['podcasts']

Run this code block if you want to clear the index, or if the index doesn't exist yet

```
# Check whether the index with the same name already exists - if so, delete it
if index_name in pinecone.list_indexes():
    pinecone.delete_index(index_name)
    
# Creates new index
pinecone.create_index(name=index_name, dimension=1536)
index = pinecone.Index(index_name=index_name)

# Confirm our index was created
pinecone.list_indexes()
```

## LLM Agent

An [LLM agent](https://python.langchain.com/en/latest/modules/agents/agents/custom_llm_agent.html) in Langchain has many configurable components, which are detailed in the Langchain documentation.

We'll employ a few of the core concepts to make an agent that talks in the way we want, can use tools to answer questions, and uses the appropriate language model to power the conversation.
- **Prompt Template:** The input template to control the LLM's behaviour and how it accepts inputs and produces outputs - this is the brain that drives your application ([docs](https://python.langchain.com/en/latest/modules/prompts/prompt_templates.html)).
- **Output Parser:** A method of parsing the output from the prompt. If the LLM produces output using certain headers, you can enable complex interactions where variables are generated by the LLM in their response and passed into the next step of the chain ([docs](https://python.langchain.com/en/latest/modules/prompts/output_parsers.html)).
- **LLM Chain:** A Chain brings together a prompt template with an LLM that will execute it - in this case we'll be using ```gpt-3.5-turbo``` but this framework can be used with OpenAI completions models, or other LLMs entirely ([docs](https://python.langchain.com/en/latest/modules/chains.html)).
- **Tool:** An external service that the LLM can use to retrieve information or execute commands should the user require it ([docs](https://python.langchain.com/en/latest/modules/agents/tools.html)).
- **Agent:** The glue that brings all of this together, an agent can call multiple LLM Chains, each with their own tools. Agents can be extended with your own logic to allow retries, error handling and any other methods you choose to add reliability to your application ([docs](https://python.langchain.com/en/latest/modules/agents.html)).

**NB:** Before using this cookbook with the Search tool you'll need to sign up on https://serpapi.com/ and generate an API key. Once you have it, store it in an environment variable named ```SERPAPI_API_KEY```

In [7]:
# Extend SerpAPIWrapper to log input and output of run
class CustomSerpAPIWrapper(SerpAPIWrapper):
    def run(self, query: str, n: int = 10, **kwargs) -> List[str]:
        print("\n\t# SerpAPIWrapper.run started")
        print(f"\n\t## Query: {query}")
        results = super().run(query=query, n=n, **kwargs)
        results_str = re.sub(r"(\[\'|\'\]|\', \')", "\n\n\t\t", str(results))
        # add an additional line break at 80 character longs if line is longer without breaking words
        results_str = re.sub(r"\b(.{50}\S*)\b", r"\1\n\t\t", results_str)
        
        print(f"\n\t## Results: {results_str}")
        print("\n\t# SerpAPIWrapper.run finished\n\n")
        return results

In [8]:
# Initiate a Search tool - note you'll need to have set SERPAPI_API_KEY as an environment variable as per the above instructions
search = CustomSerpAPIWrapper()

# Define a list of tools
tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to answer questions about current events"
    )
]

In [9]:
# Set up the prompt with input variables for tools, user input and a scratchpad for the model to record its workings
template = """Answer the following questions as best you can, but speaking as a pirate might speak. 
You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Remember to speak as a pirate when giving your final answer. Use lots of "Arg"s

Question: {input}
{agent_scratchpad}"""

In [10]:
# Set up a prompt template
class CustomPromptTemplate(BaseChatPromptTemplate):
    # The template to use
    template: str
    # The list of tools available
    tools: List[Tool]
    
    def format_messages(self, **kwargs) -> str:
        # Get the intermediate steps (AgentAction, Observation tuples)
        
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
            
        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts
        
        # Create a tools variable from the list of tools provided
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        
        # Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        formatted = self.template.format(**kwargs)
        return [HumanMessage(content=formatted)]
    
prompt = CustomPromptTemplate(
    template=template,
    tools=tools,
    # This omits the `agent_scratchpad`, `tools`, and `tool_names` variables because those are generated dynamically
    # This includes the `intermediate_steps` variable because that is needed
    input_variables=["input", "intermediate_steps"]
)

In [11]:
class CustomOutputParser(AgentOutputParser):
    
    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        
        # Check if agent should finish
        if "Final Answer:" in llm_output:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        
        # Parse out the action and action input
        regex = r"Action: (.*?)[\n]*Action Input:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        
        # If it can't parse the output it raises an error
        # You can add your own logic here to handle errors in a different way i.e. pass to a human, give a canned response
        if not match:
            raise ValueError(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        
        # Return the action and action input
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)
    
output_parser = CustomOutputParser()

In [12]:
from typing import (
    Any,
    List,
    Optional
)
from langchain.chat_models import ChatOpenAI
from langchain.schema import BaseMessage, ChatResult
from langchain_core.callbacks import CallbackManagerForLLMRun

# Create wrapper for ChatOpenAI to log input and output
class CustomChatOpenAI(ChatOpenAI):
    def _generate(self, 
        messages: List[BaseMessage],
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        stream: Optional[bool] = None,
        **kwargs: Any,
    ) -> ChatResult:
        print("\n\t# CustomChatOpenAi._generate started")
        messages_str = [json.dumps(message.dict(), indent=2) for message in messages]
        messages_str = "\n\t".join(messages_str)
        messages_str = messages_str.replace("\\n", "\n")
        messages_str = "\n\t".join(messages_str.split("\n"))
        print(f"\n\t\t## Messages: \n\t\t{messages_str}")

        result = super()._generate(messages=messages, stop=stop, run_manager=run_manager, stream=stream, **kwargs)
        result_str = json.dumps(result.dict(), indent=2)
        result_str = result_str.replace("\\n", "\n")
        result_str = "\n\t".join(result_str.split("\n"))         
        print(f"\n\t## Result: \n\t{result_str}")
        
        print("\n\t# CustomChatOpenAi._generate finished\n\n")
        return result

In [13]:
# Initiate our LLM - default is 'gpt-3.5-turbo'
llm = CustomChatOpenAI(temperature=0, verbose=True)

# LLM chain consisting of the LLM and a prompt
llm_chain = LLMChain(llm=llm, prompt=prompt)

# Using tools, the LLM chain and output_parser to make an agent
tool_names = [tool.name for tool in tools]

agent = LLMSingleActionAgent(
    llm_chain=llm_chain, 
    output_parser=output_parser,
    # We use "Observation" as our stop sequence so it will stop when it receives Tool output
    # If you change your prompt template you'll need to adjust this as well
    stop=["\nObservation:"], 
    allowed_tools=tool_names
)

In [14]:
# Initiate the agent that will respond to our queries
# Set verbose=True to share the CoT reasoning the LLM goes through
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True)

In [15]:
agent_executor.run("How many people live in canada as of 2023?")



> Entering new AgentExecutor chain...

	# CustomChatOpenAi._generate started

		## Messages: 
		{
	  "content": "Answer the following questions as best you can, but speaking as a pirate might speak. 
	You have access to the following tools:
	
	Search: useful for when you need to answer questions about current events
	
	Use the following format:
	
	Question: the input question you must answer
	Thought: you should always think about what to do
	Action: the action to take, should be one of [Search]
	Action Input: the input to the action
	Observation: the result of the action
	... (this Thought/Action/Action Input/Observation can repeat N times)
	Thought: I now know the final answer
	Final Answer: the final answer to the original input question
	
	Begin! Remember to speak as a pirate when giving your final answer. Use lots of \"Arg\"s
	
	Question: How many people live in canada as of 2023?
	",
	  "additional_kwargs": {},
	  "type": "human",
	  "example": false
	}

	## Result: 
	{
	  "gen

'Arg, there be 40 million Canadians in Canada as of 2023.'

In [16]:
agent_executor.run("How many in 2022?")



> Entering new AgentExecutor chain...

	# CustomChatOpenAi._generate started

		## Messages: 
		{
	  "content": "Answer the following questions as best you can, but speaking as a pirate might speak. 
	You have access to the following tools:
	
	Search: useful for when you need to answer questions about current events
	
	Use the following format:
	
	Question: the input question you must answer
	Thought: you should always think about what to do
	Action: the action to take, should be one of [Search]
	Action Input: the input to the action
	Observation: the result of the action
	... (this Thought/Action/Action Input/Observation can repeat N times)
	Thought: I now know the final answer
	Final Answer: the final answer to the original input question
	
	Begin! Remember to speak as a pirate when giving your final answer. Use lots of \"Arg\"s
	
	Question: How many in 2022?
	",
	  "additional_kwargs": {},
	  "type": "human",
	  "example": false
	}

	## Result: 
	{
	  "generations": [
	    {
	    

'Arrr, the year 2022 has 365 days, matey!'

## LLM Agent with History

Extend the LLM Agent with the ability to retain a [memory](https://python.langchain.com/en/latest/modules/agents/agents/custom_llm_agent.html#adding-memory) and use it as context as it continues the conversation.

We use a simple ```ConversationBufferWindowMemory``` for this example that keeps a rolling window of the last two conversation turns. LangChain has other [memory options](https://python.langchain.com/en/latest/modules/memory.html), with different tradeoffs suitable for different use cases.

In [17]:
# Set up a prompt template which can interpolate the history
template_with_history = """You are SearchGPT, a professional search engine who provides informative answers to users. 
Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Remember to give detailed, informative answers

Previous conversation history:
{history}

New question: {input}
{agent_scratchpad}"""

In [18]:
prompt_with_history = CustomPromptTemplate(
    template=template_with_history,
    tools=tools,
    # The history template includes "history" as an input variable so we can interpolate it into the prompt
    input_variables=["input", "intermediate_steps", "history"]
)

llm_chain = LLMChain(llm=llm, prompt=prompt_with_history)
tool_names = [tool.name for tool in tools]
agent = LLMSingleActionAgent(
    llm_chain=llm_chain, 
    output_parser=output_parser,
    stop=["\nObservation:"], 
    allowed_tools=tool_names
)

In [19]:
# Initiate the memory with k=2 to keep the last two turns
# Provide the memory to the agent
memory = ConversationBufferWindowMemory(k=2)
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True, memory=memory)

In [20]:
agent_executor.run("How many people live in canada as of 2023?")



> Entering new AgentExecutor chain...

	# CustomChatOpenAi._generate started

		## Messages: 
		{
	  "content": "You are SearchGPT, a professional search engine who provides informative answers to users. 
	Answer the following questions as best you can. You have access to the following tools:
	
	Search: useful for when you need to answer questions about current events
	
	Use the following format:
	
	Question: the input question you must answer
	Thought: you should always think about what to do
	Action: the action to take, should be one of [Search]
	Action Input: the input to the action
	Observation: the result of the action
	... (this Thought/Action/Action Input/Observation can repeat N times)
	Thought: I now know the final answer
	Final Answer: the final answer to the original input question
	
	Begin! Remember to give detailed, informative answers
	
	Previous conversation history:
	
	
	New question: How many people live in canada as of 2023?
	",
	  "additional_kwargs": {},
	  "type"

'As of 2023, there are approximately 40 million people living in Canada.'

In [21]:
agent_executor.run("how about in mexico?")



> Entering new AgentExecutor chain...

	# CustomChatOpenAi._generate started

		## Messages: 
		{
	  "content": "You are SearchGPT, a professional search engine who provides informative answers to users. 
	Answer the following questions as best you can. You have access to the following tools:
	
	Search: useful for when you need to answer questions about current events
	
	Use the following format:
	
	Question: the input question you must answer
	Thought: you should always think about what to do
	Action: the action to take, should be one of [Search]
	Action Input: the input to the action
	Observation: the result of the action
	... (this Thought/Action/Action Input/Observation can repeat N times)
	Thought: I now know the final answer
	Final Answer: the final answer to the original input question
	
	Begin! Remember to give detailed, informative answers
	
	Previous conversation history:
	Human: How many people live in canada as of 2023?
	AI: As of 2023, there are approximately 40 million 

'The current population of Mexico is 126.7 million.'

## Knowledge base

Create a custom vectorstore for the Agent to use as a tool to answer questions with. We'll store the results in [Pinecone](https://docs.pinecone.io/docs/quickstart), which is supported by LangChain ([Docs](https://python.langchain.com/en/latest/modules/indexes/vectorstores/examples/pinecone.html), [API reference](https://python.langchain.com/en/latest/reference/modules/vectorstore.html)). For help getting started with Pinecone or other vector databases, we have a [cookbook](https://github.com/openai/openai-cookbook/blob/colin/examples/vector_databases/Using_vector_databases_for_embeddings_search.ipynb) to help you get started.

You can check the LangChain documentation to see what other [vectorstores](https://python.langchain.com/en/latest/modules/indexes/vectorstores.html) and [databases](https://python.langchain.com/en/latest/modules/chains/examples/sqlite.html) are available.

For this example we'll use the transcripts of the Stuff You Should Know podcast, which was provided thanks to OSF DOI [10.17605/OSF.IO/VM9NT](https://doi.org/10.17605/OSF.IO/VM9NT)

In [ ]:
import wget

# Here is a URL to a zip archive containing the transcribed podcasts
# Note that this data has already been split into chunks and embeddings from OpenAI's text-embedding-ada-002 embedding model are included
content_url = 'https://cdn.openai.com/API/examples/data/sysk_podcast_transcripts_embedded.json.zip'

# Download the file (it is ~541 MB so this will take some time)
wget.download(content_url)

In [ ]:
# Load podcasts
with zipfile.ZipFile("sysk_podcast_transcripts_embedded.json.zip","r") as zip_ref:
    zip_ref.extractall("./data")


In [22]:
import ijson

processed_podcasts = []

file_path = './data/sysk_podcast_transcripts_embedded.json'
progress_lines = tqdm(total=38145, unit='lines')

with open(file_path, 'r') as f:
    items = ijson.items(f, 'item')
    while True:
        try:
            item = next(items)
            processed_podcasts.append(item)
            progress_lines.update()
        except StopIteration as e:
            break



  0%|          | 0/38145 [00:00<?, ?lines/s]

In [23]:
# Have a look at the contents
pd.DataFrame(processed_podcasts).head()

,id,filename,title,url,text_chunk,embedding,cleaned_id
0,sysk_with_transcripts_SYSK Selects How Crime S...,sysk_with_transcripts_SYSK Selects How Crime S...,\n\nSYSK Selects How Crime Scene Cleanup Works,https://chtbl.com/track/5899E/podtrac.com/pts/...,Title: sysk_with_transcripts_SYSK Selects How ...,"[0.021279960870742798, -0.005817972123622894, ...",sysk_with_transcripts_SYSK Selects How Crime S...
1,sysk_with_transcripts_SYSK Selects How Crime S...,sysk_with_transcripts_SYSK Selects How Crime S...,\n\nSYSK Selects How Crime Scene Cleanup Works,https://chtbl.com/track/5899E/podtrac.com/pts/...,Title: sysk_with_transcripts_SYSK Selects How ...,"[0.013859338127076626, 0.00857278611510992, 0....",sysk_with_transcripts_SYSK Selects How Crime S...
2,sysk_with_transcripts_SYSK Selects How Crime S...,sysk_with_transcripts_SYSK Selects How Crime S...,\n\nSYSK Selects How Crime Scene Cleanup Works,https://chtbl.com/track/5899E/podtrac.com/pts/...,Title: sysk_with_transcripts_SYSK Selects How ...,"[0.015242221765220165, 0.016030369326472282, 0...",sysk_with_transcripts_SYSK Selects How Crime S...
3,sysk_with_transcripts_SYSK Selects How Crime S...,sysk_with_transcripts_SYSK Selects How Crime S...,\n\nSYSK Selects How Crime Scene Cleanup Works,https://chtbl.com/track/5899E/podtrac.com/pts/...,Title: sysk_with_transcripts_SYSK Selects How ...,"[0.004371842369437218, -0.003036574460566044, ...",sysk_with_transcripts_SYSK Selects How Crime S...
4,sysk_with_transcripts_SYSK Selects How Crime S...,sysk_with_transcripts_SYSK Selects How Crime S...,\n\nSYSK Selects How Crime Scene Cleanup Works,https://chtbl.com/track/5899E/podtrac.com/pts/...,Title: sysk_with_transcripts_SYSK Selects How ...,"[0.017309172078967094, 0.015154214575886726, 0...",sysk_with_transcripts_SYSK Selects How Crime S...


In [ ]:
# Add the text embeddings to Pinecone

batch_size = 100  # how many embeddings we create and insert at once

index = pinecone.Index(index_name)

def to_float(x):
    return [float(y) for y in x]

for i in tqdm(range(0, len(processed_podcasts), batch_size)):
    # find end of batch
    i_end = min(len(processed_podcasts), i+batch_size)
    meta_batch = processed_podcasts[i:i_end]
    # get ids
    ids_batch = [x['cleaned_id'] for x in meta_batch]
    # get texts to encode
    texts = [x['text_chunk'] for x in meta_batch]
    # add embeddings
    embeds = [to_float(x['embedding']) for x in meta_batch]
    # cleanup metadata
    meta_batch = [{
        'filename': x['filename'],
        'title': x['title'],
        'text_chunk': x['text_chunk'],
        'url': x['url']
    } for x in meta_batch]
    to_upsert = list(zip(ids_batch, embeds, meta_batch))
    # upsert to Pinecone
    index.upsert(vectors=to_upsert)

In [25]:
# Configuring the embeddings to be used by our retriever to be OpenAI Embeddings, matching our embedded corpus
embeddings = OpenAIEmbeddings()


# Loads a docsearch object from an existing Pinecone index so we can retrieve from it
docsearch = Pinecone.from_existing_index(index_name,embeddings,text_key='text_chunk')

In [26]:
retriever = docsearch.as_retriever()

In [27]:
query_docs = retriever.get_relevant_documents("can you live without a bank account")

In [28]:
# Print out the title and content for the most relevant retrieved documents
print("\n".join(['Title: ' + x.metadata['title'].strip() + '\n\n' + x.page_content + '\n\n' for x in query_docs]))

Title: sysk: Can You Live Without a Bank Account?

Title: sysk_with_transcripts_Can you live without a bank account.json;  And if you had a life, you didn't necessarily rectify your bank checkbook every day. Oh, wait, what is balancing a checkbook mean? Seriously? Yeah. Thank God for my wife. So another reason you might avoid a bank is philosophically. There may be a longstanding distrust of banks in your family that you don't want to put your money in, or you may just want to be like, you know what? I don't want to take part in this modern society. I want to kind of drop out a bit. And a really good first move is to shut your bank account down. That's a big statement. Oh, yeah, it is. But a lot of people that are underbanked and don't have accounts aren't there on purpose. It's not some philosophical statement. A lot of times it's simply because they are poor and they don't have a lot of alternatives. Yeah. And the other thing about not having a bank account, not only do you not have 

## LLM Agent with Tools

Extend our list of tools by creating a [RetrievalQA](https://python.langchain.com/en/latest/modules/chains/index_examples/vector_db_qa.html) chain leveraging our Pinecone knowledge base.

In [32]:
from langchain.chains import RetrievalQA

retrieval_llm = OpenAI(temperature=0)

podcast_retriever = RetrievalQA.from_chain_type(llm=retrieval_llm, chain_type="stuff", retriever=docsearch.as_retriever())

In [33]:
expanded_tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to answer questions about current events"
    ),
    Tool(
        name = 'Knowledge Base',
        func=podcast_retriever.run,
        description="Useful for general questions about how to do things and for details on interesting topics. Input should be a fully formed question."
    )
]

In [34]:
# Re-initialize the agent with our new list of tools
prompt_with_history = CustomPromptTemplate(
    template=template_with_history,
    tools=expanded_tools,
    input_variables=["input", "intermediate_steps", "history"]
)
llm_chain = LLMChain(llm=llm, prompt=prompt_with_history)
multi_tool_names = [tool.name for tool in expanded_tools]
multi_tool_agent = LLMSingleActionAgent(
    llm_chain=llm_chain, 
    output_parser=output_parser,
    stop=["\nObservation:"], 
    allowed_tools=multi_tool_names
)

In [35]:
multi_tool_memory = ConversationBufferWindowMemory(k=2)
multi_tool_executor = AgentExecutor.from_agent_and_tools(agent=multi_tool_agent, tools=expanded_tools, verbose=True, memory=multi_tool_memory)

In [36]:
multi_tool_executor.run("Hi, I'd like to know how you can live without a bank account")



> Entering new AgentExecutor chain...

	# CustomChatOpenAi._generate started

		## Messages: 
		{
	  "content": "You are SearchGPT, a professional search engine who provides informative answers to users. 
	Answer the following questions as best you can. You have access to the following tools:
	
	Search: useful for when you need to answer questions about current events
	Knowledge Base: Useful for general questions about how to do things and for details on interesting topics. Input should be a fully formed question.
	
	Use the following format:
	
	Question: the input question you must answer
	Thought: you should always think about what to do
	Action: the action to take, should be one of [Search, Knowledge Base]
	Action Input: the input to the action
	Observation: the result of the action
	... (this Thought/Action/Action Input/Observation can repeat N times)
	Thought: I now know the final answer
	Final Answer: the final answer to the original input question
	
	Begin! Remember to give de

'Some alternatives to having a bank account include using prepaid credit cards, hosting on Airbnb, and using alternative currencies. However, living without a bank account can be challenging and comes with certain risks and limitations.'

In [37]:
multi_tool_executor.run('Can you tell me some interesting facts about whether zoos are good or bad for animals')



> Entering new AgentExecutor chain...

	# CustomChatOpenAi._generate started

		## Messages: 
		{
	  "content": "You are SearchGPT, a professional search engine who provides informative answers to users. 
	Answer the following questions as best you can. You have access to the following tools:
	
	Search: useful for when you need to answer questions about current events
	Knowledge Base: Useful for general questions about how to do things and for details on interesting topics. Input should be a fully formed question.
	
	Use the following format:
	
	Question: the input question you must answer
	Thought: you should always think about what to do
	Action: the action to take, should be one of [Search, Knowledge Base]
	Action Input: the input to the action
	Observation: the result of the action
	... (this Thought/Action/Action Input/Observation can repeat N times)
	Thought: I now know the final answer
	Final Answer: the final answer to the original input question
	
	Begin! Remember to give de

ValueError: Could not parse LLM output: `Based on the information I found, the final answer is that the question of whether zoos are good or bad for animals is complex and depends on various factors. While zoos can provide educational opportunities, protected environments for endangered species, and contribute to conservation efforts, they also have drawbacks. Animals in zoos may experience poor living conditions, limited space, and lack of stimulation. Some zoos still obtain animals from the wild, which can harm threatened species. Additionally, animals in zoos often have shorter lifespans compared to those in the wild. Therefore, it is important to consider both the positive and negative aspects when evaluating the impact of zoos on animals.`

You now have a template to deploy conversational agents with tools. If you want to extend this with a Custom Agent to add your own retry behaviour or treatment of input/output variables, then follow [this article](https://python.langchain.com/en/latest/modules/agents/agents/custom_agent.html).

We look forward to seeing what you build!